In [ ]:
# Import libraries
import pandas as pd
import numpy as np

from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import warnings
warnings.filterwarnings('ignore')

# **1. Data Cleaning and Preparation**

In [ ]:
data = pd.read_csv('../data/online_retail.csv')
print(f'Rows, Columns count', data.shape)
data.head()

**1. Data Inspection**

In [ ]:
# Data summary information
data.info()

Missing values are present in `Description, CustomerID`, `InvoiceDate, CustomerID` have incorrect datatypes

In [ ]:
# Statistical summary
data.describe()

Our numerical columns are highly skewed, they also appear to have invalid values; there are negative values present

**2. Data Cleaning**

In [ ]:
# Data cleaning
# Check for and drop missing values for critical columns
print(f'Missing values per column\n',data.isnull().sum())
data.dropna(subset=['InvoiceDate', 'CustomerID', 'InvoiceDate'], inplace=True)

# Change invoiceDate to DateTime format
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors='coerce')
# Convert customerID to str
data['CustomerID'] = data['CustomerID'].astype(int).astype(str)

# Filter invalid values in Quantity and UnitPrice columns
data = data[(data['Quantity'] > 0) & (data['UnitPrice'] > 0.01)]

# Create TransactionAmount columns
data['TransactionAmount'] = (data['UnitPrice'] * data['Quantity']).round(2)

# Inspection check
print(f'Missing values after cleaning:\n', data.isna().sum(), '\n')
print(f'Updated Rows, Columns:', data.shape)
data.head()

**Checking Duplicate Entries**

In [ ]:
print(f'Duplicate Entries: ', data.duplicated().sum()) # 5192 Duplicates
duplicates = data[data.duplicated(keep=False)]
duplicates.head() 

In [ ]:
duplicate_groups = data[data.duplicated(keep=False)].groupby(list(data.columns)).size()
print(len(duplicate_groups))
duplicate_groups.head()

In [ ]:
# Drop duplicates but keep 1st occurence of a duplicated entry
data = data.drop_duplicates(keep='last')
print(f'Duplicated Entries: ', data.duplicated().sum())

In [ ]:
data.describe(include=['number'])

In [ ]:
print(data[['Quantity', 'TransactionAmount']].skew())

**Handling Outliers**

In [ ]:
def cap_outliers(series, lower_quantile=0.01, upper_quantile=0.99):
    lower_bound = series.quantile(lower_quantile)
    upper_bound = series.quantile(upper_quantile)
    return series.clip(lower=lower_bound, upper=upper_bound)

# # Automatically apply capping to all numeric columns
# data[data.select_dtypes(include=['float64', 'int64']).columns] = data.select_dtypes(include=['float64', 'int64']).apply(
#     lambda x: cap_outliers(x)
# )
# Columns to cap outliers
columns_to_cap = ['Quantity', 'TransactionAmount']  # Replace with your column names

# Apply the capping function to each column
for col in columns_to_cap:
    data[col] = cap_outliers(data[col])

# Check summary statistics to confirm
data.describe(include='number')

In [ ]:
data.head(3)

In [ ]:
print(f'Skewness before Log-Transform after removing Outliers: \n', data[['Quantity', 'TransactionAmount']].skew(),'\n')
# print(f'Skewness after Log-Transform: \n', data[['Quantity_log', 'TransactionAmount_log']].skew())

In [ ]:
# Duplicates check
print(f'Number of Duplicates:', data.duplicated().sum(), '\n')
# Drop duplicates
data.drop_duplicates(keep='first', inplace=True)
# Inpect duplicate entries (Should return zero entries)
data[data.duplicated(keep=False)]

In [ ]:
data.info()

In [ ]:
# Save cleaned data
data.to_pickle('../data/cleaned_data.pkl')

## **Exploratory Data Analysis**

In [ ]:
df = pd.read_pickle('../data/cleaned_data.pkl')
df.head()

---

### **Calculate RFM Metrics**

In [ ]:
# Calculate Reference data
reference_date = df['InvoiceDate'].max() + pd.DateOffset(1)
reference_date

In [ ]:
# Calculate RFM Metrics
rfm = df.groupby('CustomerID').agg(
    Recency=('InvoiceDate', lambda x: (reference_date - x.max()).days),
    Frequency=('InvoiceNo', 'nunique'),
    Monetary=('TransactionAmount', 'sum')
).reset_index()

# RFM table preview
rfm.head()

In [ ]:
# Check skewness of RFM metrics
print(rfm[["Recency", "Frequency", "Monetary"]].skew())

In [ ]:
# Log-Transform RFM Metrics
rfm[["Recency_log", "Frequency_log", "Monetary_log"]] = rfm[["Recency", "Frequency", "Monetary"]].apply(lambda x: np.log1p(x)).round(2)
rfm.head()

In [ ]:
rfm[["Recency_log", "Frequency_log", "Monetary_log"]].skew()

### **RFM Scoring**

In [ ]:
rfm['Recency_log'] = pd.qcut(rfm['Recency_log'], 5, labels=[5,4,3,2,1])
rfm['Frequency_log'] = pd.qcut(rfm['Frequency_log'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['Monetary_log'] = pd.qcut(rfm['Monetary_log'].rank(method='first'), 5, labels=[1,2,3,4,5])

# Combine scores into one
rfm['RFM_score'] = rfm['Recency_log'].astype(int) + rfm['Frequency_log'].astype(int) + rfm['Monetary_log'].astype(int)

rfm.head()

### **Value Segmentation**

In [ ]:
value_labels = ['Low-value', 'Mid-value', 'High-value']
rfm['Value_segment'] = pd.qcut(rfm['RFM_score'], q=3, labels=value_labels)
# Preview
rfm.head()

### **Behavioral Segmentation**

In [ ]:
def behaviour_segment(row):
    if row['RFM_score'] >= 13:
        return 'VIP'
    if row['RFM_score'] >= 10:
        return 'Loyal Customers'
    if row['RFM_score'] >= 7:
        return 'Potential Loyalists'
    if row['RFM_score'] >= 5:
        return 'At Risk'
    else:
        return 'Hibernating'
    
# Applying the segmentation strategy
rfm['Customer_segment'] = rfm.apply(behaviour_segment, axis=1)
print(rfm['Customer_segment'].value_counts())

rfm.head()

In [ ]:
# Merge the RFM metrics back to the original dataframe
df_combined = df.merge(rfm, on='CustomerID', how='left')
print(df_combined.shape)
df_combined.head(3)

In [ ]:
# Save combined DataFrame
df_combined.to_pickle('../data/combined_data.pkl')

## **Overview of Segments (RFM Analysis)**

In [ ]:
# Load combined DataFrame
rfm_df = pd.read_pickle('../data/combined_data.pkl')
rfm_df.columns

How are the segments distributed?

In [ ]:
# Value segments
value_counts = rfm_df['Value_segment'].value_counts()
print(value_counts)

# Plot distribution
sns.countplot(data=rfm_df, x='Value_segment', order=value_counts.index, palette="viridis")
plt.title('Distribution of Value Segments')
plt.xlabel('Value Segment')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()